In [ ]:
!pip install python-mnist

In [ ]:
!pip install emnist

In [ ]:
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

from mnist import MNIST

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from emnist import extract_training_samples
from emnist import list_datasets
from emnist import extract_test_samples

In [ ]:
# list_datasets()

['balanced', 'byclass', 'bymerge', 'digits', 'letters', 'mnist']

In [ ]:
train_images, train_labels = extract_training_samples('byclass')
# print(train_images.shape)
# print(train_labels.shape)

In [ ]:
test_images, test_labels = extract_test_samples('byclass')
# print(test_images.shape)
# print(test_labels.shape)

In [ ]:
X_train = []
y_train = train_labels.copy()
X_test = []
y_test = test_labels.copy()

In [ ]:
def reprocess(image):
    image = cv2.resize(image, (32, 32), interpolation=cv2.INTER_AREA)
    ret, image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = image.astype('float32')
    image/=255

    return image

In [ ]:
for idx, img in enumerate(train_images):
    img = reprocess(train_images[idx])
    X_train.append(img)

In [ ]:
X_train = np.array(X_train)
X_train = np.expand_dims(X_train, axis=3)
del train_images
del train_labels

In [ ]:
for idx, img in enumerate(test_images):
    img = reprocess(test_images[idx])
    X_test.append(img)

In [ ]:
X_test = np.array(X_test)
X_test = np.expand_dims(X_test, axis=3)
del test_images
del test_labels

In [ ]:
X_train.shape

(697932, 32, 32, 1)

In [ ]:
X_test.shape

(116323, 32, 32, 1)

In [ ]:
# X_train = np.expand_dims(X_train, axis=3)
# X_test = np.expand_dims(X_test, axis=3)

In [ ]:
# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')

In [ ]:
# train_images = tf.keras.utils.normalize(train_images, axis = 1)
# test_images = tf.keras.utils.normalize(test_images, axis = 1)

In [ ]:
# train_images = np.expand_dims(train_images, axis=3)
# test_images = np.expand_dims(test_images, axis=3)

In [ ]:
# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
# model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(2,2))

# model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(2,2))

# model.add(tf.keras.layers.Dropout(0.2))

# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(256, activation='relu'))
# model.add(tf.keras.layers.Dense(128, activation='relu'))
# model.add(tf.keras.layers.Dense(62, activation = 'softmax'))

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), input_shape=(32, 32, 1), padding='same'))
model.add(tf.keras.layers.BatchNormalization(axis=3))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(32, (3,3)))
model.add(tf.keras.layers.BatchNormalization(axis=3))
model.add(tf.keras.layers.Activation('relu'))
# model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Conv2D(64, (3,3)))
model.add(tf.keras.layers.BatchNormalization(axis=3))
model.add(tf.keras.layers.Activation('relu'))
# model.add(tf.keras.layers.MaxPooling2D(2,2))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(62, activation = 'softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
21811/21811 [==============================] - 1170s 53ms/step - loss: 0.4906 - accuracy: 0.8304
Epoch 2/10
21811/21811 [==============================] - 1156s 53ms/step - loss: 0.3962 - accuracy: 0.8558
Epoch 3/10
21811/21811 [==============================] - 1154s 53ms/step - loss: 0.3756 - accuracy: 0.8617
Epoch 4/10
21811/21811 [==============================] - 1157s 53ms/step - loss: 0.3628 - accuracy: 0.8653
Epoch 5/10
21811/21811 [==============================] - 1146s 53ms/step - loss: 0.3529 - accuracy: 0.8675
Epoch 6/10
21811/21811 [==============================] - 1144s 52ms/step - loss: 0.3452 - accuracy: 0.8700
Epoch 7/10
21811/21811 [==============================] - 1148s 53ms/step - loss: 0.3381 - accuracy: 0.8721
Epoch 8/10
21811/21811 [==============================] - 1157s 53ms/step - loss: 0.3324 - accuracy: 0.8737
Epoch 9/10
21811/21811 [==============================] - 1151s 53ms/step - loss: 0.3263 - accuracy: 0.8757
Epoch 10/10
21811/21811 [===

In [ ]:
model.evaluate(X_test, y_test)

3636/3636 [==============================] - 50s 14ms/step - loss: 0.3885 - accuracy: 0.8637


[0.3885347545146942, 0.8636898994445801]

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
  model.save_weights("/content/drive/MyDrive/character_reader2.h5")
  print("Saved model to disk")

Saved model to disk


In [ ]:
from keras.models import Model

In [ ]:
from tensorflow.keras.models.Model import load_weights

In [ ]:
# model = emotion_recognition((48,48,1))
# adam = keras.optimizers.Adam(learning_rate=0.001)
# model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights('/content/drive/MyDrive/character_reader.h5')